### Shortest path (BFS):

###### In this notebook Spark is used to construct the shortest path between two vertices s, f: The result is a sequence of vertices.

In [1]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local"))

In [2]:
def parse_edge(s):
    user, follower = s.split("\t")
    return (int(user), int(follower))

def step(item):
    prev_v, prev_d, next_v = item[0], item[1][0], item[1][1]
    return (next_v, prev_v)

def complete(item):
    v, old_d, new_d = item[0], item[1][0], item[1][1]
    return (v, old_d if old_d is not None else new_d)

# def step_path(item):
#     prev_v, path, next_v = item[0], item[1][0], item[1][1]
#     path += tuple([next_v])
#     return (next_v, path)

In [3]:
def path_step(item):
    prev_v, path, next_v = item[0], item[1][0], item[1][1]
    prev_v = next_v
    path += tuple([next_v])
    return (prev_v,path)

In [5]:
n = 4
edges = sc.textFile("/data/twitter/twitter_sample_small.txt").map(parse_edge).cache()
forward_edges = edges.map(lambda e: (e[1], e[0])).partitionBy(n).persist()

s = 12
f = 34

paths = sc.parallelize([ (s, tuple([s])) ]).partitionBy(n)
    
while True:
    paths = paths.join(forward_edges, n).map(path_step)
    count = paths.filter(lambda p: p[0] == f).count()
    if count > 0:
        break
    
ans = paths.filter(lambda p: p[0] == f).collect()


##Print the result
print(' -> '.join(map(str, ans[0][1])))

12 -> 422 -> 53 -> 52 -> 107 -> 20 -> 23 -> 274 -> 34


In [10]:
# t = f
# ans = []
# ans.append(t)
# c = 0
# while t != 0:
#     for i in l:
#         if i[0] == t:
#             t = i[1]
#             ans.append(t)

# nl = ans[::-1]
# for el in range(1,len(nl)):
#     print(nl[el], end = "")    
#     if(el < len(nl)-1):
#         print(",", end = "")